In [2]:
# === Compare-1: 두 calib.npz 요약 비교 ===
import os, numpy as np, cv2, pathlib, math, json

CAL_A = r"chess_try_0/calib_out/calib.npz"   # (예) 추가 전
CAL_B = r"chess_try_1/calib_out/calib.npz"   # (예) 추가 후

def load_cal(p):
    cal = np.load(p, allow_pickle=True)
    d = {
        "path": p,
        "model": str(cal["model"]),
        "rms": float(cal["rms"]),
        "K": cal["K"],
        "D": cal["D"],
        "newK": cal["newK"],
        "map1": cal["map1"],
        "map2": cal["map2"],
        "img_size": tuple(map(int, cal["img_size"])),
        "pattern": tuple(map(int, cal["pattern"])),
        "used": list(map(str, cal.get("used_images", [])))
    }
    return d

def fov_deg(newK, size):  # rectified 카메라의 유사 fov
    W,H = size
    fx, fy = newK[0,0], newK[1,1]
    fovx = 2*math.degrees(math.atan(0.5*W/fx))
    fovy = 2*math.degrees(math.atan(0.5*H/fy))
    return fovx, fovy

A = load_cal(CAL_A)
B = load_cal(CAL_B)

def summarize(tag, C):
    W,H = C["img_size"]
    fx, fy = C["newK"][0,0], C["newK"][1,1]
    cx, cy = C["newK"][0,2], C["newK"][1,2]
    fovx, fovy = fov_deg(C["newK"], C["img_size"])
    print(f"\n[{tag}] {C['path']}")
    print(f"  model={C['model']}, RMS={C['rms']:.4f}, img_size={W}x{H}, pattern={C['pattern']}")
    print(f"  newK fx,fy=({fx:.1f},{fy:.1f}), cx,cy=({cx:.1f},{cy:.1f})"
          f"  | norm(D)={float(np.linalg.norm(C['D'].ravel())):.4f}")
    print(f"  effective FOV ≈ {fovx:.1f}° x {fovy:.1f}°")
    print(f"  used_images={len(C['used'])}")

summarize("A(old)", A)
summarize("B(new)", B)

# 사용 이미지 겹침
overlap = len(set(A["used"]).intersection(set(B["used"])))
print(f"\n[used overlap] {overlap} common of A:{len(A['used'])}, B:{len(B['used'])}")

# img_size 불일치 경고
if A["img_size"] != B["img_size"]:
    print("\n[WARN] img_size가 다릅니다. 비교/미리보기 시 각 사이즈에 맞게 맵을 재생성해서 쓸게요.")



[A(old)] chess_try_0/calib_out/calib.npz
  model=fisheye, RMS=0.1465, img_size=2592x1944, pattern=(9, 6)
  newK fx,fy=(971.4,971.2), cx,cy=(1319.8,890.5)  | norm(D)=0.0358
  effective FOV ≈ 106.3° x 90.0°
  used_images=40

[B(new)] chess_try_1/calib_out/calib.npz
  model=fisheye, RMS=0.2635, img_size=2592x1944, pattern=(9, 6)
  newK fx,fy=(968.1,968.3), cx,cy=(1311.3,892.0)  | norm(D)=0.0306
  effective FOV ≈ 106.5° x 90.2°
  used_images=87

[used overlap] 40 common of A:40, B:87


In [3]:
# === Compare-2: 유효화면(Valid area) 비율 비교 ===
import cv2, numpy as np

def valid_ratio_from_map(m1, m2, size):
    W,H = size
    src = np.ones((H,W), np.uint8)*255
    dst = cv2.remap(src, m1, m2, cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    ratio = float((dst>0).mean())
    return ratio

va = valid_ratio_from_map(A["map1"], A["map2"], A["img_size"])
vb = valid_ratio_from_map(B["map1"], B["map2"], B["img_size"])
print(f"A valid area: {va*100:.1f}%")
print(f"B valid area: {vb*100:.1f}%")


A valid area: 100.0%
B valid area: 100.0%


In [6]:
# === Compare-3: A vs B 보정 결과 나란히 저장 ===
import glob, cv2, numpy as np, pathlib, os

TEST_DIR = r"chess_try_1/test_0"   # 비교에 쓸 일반 사진 폴더 (없으면 used_images에서 뽑음)
OUT_DIR  = pathlib.Path("compare_AB")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 테스트 이미지 선정
paths = sorted(glob.glob(os.path.join(TEST_DIR, "*.jpg")) + glob.glob(os.path.join(TEST_DIR, "*.png")))
if not paths:
    # A 또는 B의 used_images에서 6장 선택
    pool = [p for p in (A["used"]+B["used"]) if os.path.exists(p)]
    paths = sorted(list(dict.fromkeys(pool)))[:6]
print("test images:", len(paths))

def make_map_for_size(C, size):
    w,h = size
    if C["img_size"] == (w,h):
        return C["map1"], C["map2"]
    # 사이즈 다르면 모델별로 재생성 (fisheye는 balance≈0.0에 해당)
    K,D = C["K"], C["D"]
    if C["model"]=="pinhole":
        newK,_ = cv2.getOptimalNewCameraMatrix(K,D,(w,h),0,(w,h))
        m1,m2 = cv2.initUndistortRectifyMap(K,D,None,newK,(w,h),cv2.CV_16SC2)
    else:
        R = np.eye(3)
        newK = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K,D,(w,h),R,balance=0.0)
        m1,m2 = cv2.fisheye.initUndistortRectifyMap(K,D,R,newK,(w,h),cv2.CV_16SC2)
    return m1,m2

def sbs(*imgs, maxw=2200):
    h = min(i.shape[0] for i in imgs)
    rs = [cv2.resize(i, (int(i.shape[1]*h/i.shape[0]), h)) for i in imgs]
    cat = np.hstack(rs)
    if cat.shape[1] > maxw:
        cat = cv2.resize(cat, (maxw, int(maxw*cat.shape[0]/cat.shape[1])))
    return cat

for p in paths:
    img = cv2.imread(p, cv2.IMREAD_COLOR)
    if img is None: 
        print("read fail:", p); 
        continue
    h,w = img.shape[:2]
    m1a,m2a = make_map_for_size(A, (w,h))
    m1b,m2b = make_map_for_size(B, (w,h))
    uda = cv2.remap(img, m1a, m2a, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    udb = cv2.remap(img, m1b, m2b, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    cv2.putText(uda, "A", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2, cv2.LINE_AA)
    cv2.putText(udb, "B", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2, cv2.LINE_AA)

    comp = sbs(img, uda, udb)
    out = OUT_DIR / (pathlib.Path(p).stem + "_A_B.jpg")
    cv2.imwrite(str(out), comp)
    print("wrote:", out)
print("done →", OUT_DIR.resolve())


test images: 7
wrote: compare_AB\img_t+90_p+030_20250904_200519_802_A_B.jpg
wrote: compare_AB\img_t+90_p+060_20250904_200236_757_A_B.jpg
wrote: compare_AB\img_t+90_p+060_20250904_200750_383_A_B.jpg
wrote: compare_AB\img_t+90_p+090_20250904_200237_844_A_B.jpg
wrote: compare_AB\img_t+90_p+090_20250904_200524_040_A_B.jpg
wrote: compare_AB\img_t+90_p+120_20250904_200238_931_A_B.jpg
wrote: compare_AB\img_t+90_p+120_20250904_200526_231_A_B.jpg
done → C:\Users\gmlwn\OneDrive\바탕 화면\ICon1학년\광통신\PTCamera_waveshare\test_set\compare_AB


In [7]:
# === Compare-4: 직선성(RMSE) 비교 (원본→A, 원본→B) ===
import numpy as np, cv2, pathlib

def straightness_rmse(bgr, side="L", band=0.10):
    h,w = bgr.shape[:2]
    x0,x1 = (0, int(w*band)) if side=="L" else (int(w*(1-band)), w)
    roi = bgr[:, x0:x1]
    g = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    e = cv2.Canny(g, 60, 180)
    ys, xs = np.where(e>0)
    if len(xs) < 200:
        return None
    xs = xs + x0
    A = np.vstack([ys, np.ones_like(ys)]).T
    a,b = np.linalg.lstsq(A, xs, rcond=None)[0]  # x = a*y + b
    xs_fit = a*ys + b
    rmse = float(np.sqrt(np.mean((xs - xs_fit)**2)))
    return rmse

def make_map(C, size):
    w,h = size
    if C["img_size"] == (w,h):
        return C["map1"], C["map2"]
    K,D = C["K"], C["D"]
    if C["model"]=="pinhole":
        newK,_ = cv2.getOptimalNewCameraMatrix(K,D,(w,h),0,(w,h))
        return cv2.initUndistortRectifyMap(K,D,None,newK,(w,h),cv2.CV_16SC2)
    else:
        R = np.eye(3)
        newK = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K,D,(w,h),R,balance=0.0)
        return cv2.fisheye.initUndistortRectifyMap(K,D,R,newK,(w,h),cv2.CV_16SC2)

for p in paths[:12]:
    img = cv2.imread(p)
    if img is None: continue
    h,w = img.shape[:2]
    m1a,m2a = make_map(A, (w,h))
    m1b,m2b = make_map(B, (w,h))
    uda = cv2.remap(img, m1a, m2a, cv2.INTER_LINEAR)
    udb = cv2.remap(img, m1b, m2b, cv2.INTER_LINEAR)
    L0 = straightness_rmse(img,"L"); R0 = straightness_rmse(img,"R")
    La = straightness_rmse(uda,"L"); Ra = straightness_rmse(uda,"R")
    Lb = straightness_rmse(udb,"L"); Rb = straightness_rmse(udb,"R")

    def fmt(a,b): 
        return "n/a" if a is None or b is None else f"{a:.2f}->{b:.2f} (Δ{(b-a):+.2f})"
    print(pathlib.Path(p).name,
          " L A:", fmt(L0,La), " B:", fmt(L0,Lb),
          " | R A:", fmt(R0,Ra), " B:", fmt(R0,Rb))


img_t+90_p+030_20250904_200519_802.jpg  L A: 78.75->48.93 (Δ-29.82)  B: 78.75->47.85 (Δ-30.90)  | R A: 65.89->65.13 (Δ-0.76)  B: 65.89->65.75 (Δ-0.15)
img_t+90_p+060_20250904_200236_757.jpg  L A: 66.09->67.69 (Δ+1.60)  B: 66.09->67.37 (Δ+1.28)  | R A: 61.22->58.06 (Δ-3.17)  B: 61.22->57.99 (Δ-3.23)
img_t+90_p+060_20250904_200750_383.jpg  L A: 72.46->57.88 (Δ-14.57)  B: 72.46->57.78 (Δ-14.67)  | R A: 73.13->70.62 (Δ-2.50)  B: 73.13->71.10 (Δ-2.03)
img_t+90_p+090_20250904_200237_844.jpg  L A: 73.03->66.17 (Δ-6.86)  B: 73.03->67.22 (Δ-5.81)  | R A: 64.55->63.32 (Δ-1.23)  B: 64.55->63.20 (Δ-1.35)
img_t+90_p+090_20250904_200524_040.jpg  L A: 66.01->67.62 (Δ+1.61)  B: 66.01->68.72 (Δ+2.72)  | R A: 73.02->74.09 (Δ+1.07)  B: 73.02->74.07 (Δ+1.04)
img_t+90_p+120_20250904_200238_931.jpg  L A: 55.96->74.71 (Δ+18.75)  B: 55.96->74.83 (Δ+18.87)  | R A: 55.73->74.73 (Δ+19.00)  B: 55.73->74.96 (Δ+19.23)
img_t+90_p+120_20250904_200526_231.jpg  L A: 81.81->68.51 (Δ-13.30)  B: 81.81->63.54 (Δ-18.27)  | 